In [121]:
import pandas as pd
import numpy as np
from datetime import datetime
from prefect.blocks.system import Secret
from catnip.fla_redshift import FLA_Redshift
from typing import Dict
from concurrent.futures import ThreadPoolExecutor
import openpyxl 
from catnip.fla_sharepoint import FLA_Sharepoint

In [145]:
nightly = pd.read_excel("C:\\Users\\riffere\\Florida Panthers\\SP-BS - Documents\Data Science\Resources\\2023-24 Temporary Report Data\\23-24 Panthers Nightly Data TSS Temp.xlsx")
plans = pd.read_excel("C:\\Users\\riffere\\Florida Panthers\\SP-BS - Documents\Data Science\Resources\\2023-24 Temporary Report Data\\23-24 Panthers Subscription Data TSS Temp.xlsx")

In [146]:
df = nightly.groupby(by = ['Plan Type'])[['Gross Revenue','Net Revenue','Tickets']].sum()
df.reset_index(inplace = True)
df = df[df['Plan Type'].apply(lambda x: len(str(x))>1)]
df['FSE'] = 0
df1 = nightly.groupby(by = ['Plan Type'])['Customer ID'].nunique()
df1 = df1.to_frame()
df1.reset_index(inplace = True)
df = df.merge(right = df1, how = 'left', on = ['Plan Type'])
df['Gross Paid ATP'] = [x/y for x,y in zip(df['Gross Revenue'],df['Tickets'])]
df['Net Paid ATP'] = [x/y for x,y in zip(df['Net Revenue'],df['Tickets'])]
df = df[['Plan Type', 'Gross Revenue', 'Net Revenue', 'FSE', 'Tickets','Customer ID', 'Gross Paid ATP',
          'Net Paid ATP']].rename(columns = {'Tickets' : 'Paid Seats', 'Customer ID' : 'Unique Accounts'})

In [147]:
df2 = plans.groupby(by = ['Plan Type', 'PC2'])[['Gross Revenue','Net Revenue','Tickets']].sum()
df2.reset_index(inplace = True)
df2 = df2[df2['Plan Type'].apply(lambda x: len(str(x))>1)]
df2['Paid Seats'] = [x*y for x,y in zip([42,42,25,42,25,42], df2['Tickets'])]
df2 = df2.groupby(by = ['Plan Type'])[['Gross Revenue','Net Revenue','Tickets', 'Paid Seats']].sum()
df2.reset_index(inplace = True)
df3 = plans.groupby(by = ['Plan Type'])['Customer ID'].nunique()
df3 = df3.to_frame()
df3.reset_index(inplace = True)
df2 = df2.merge(right = df3, how = 'left', on = ['Plan Type'])
df2['Gross Paid ATP'] = [x/y for x,y in zip(df2['Gross Revenue'],df2['Paid Seats'])]
df2['Net Paid ATP'] = [x/y for x,y in zip(df2['Net Revenue'],df2['Paid Seats'])]
df2 = df2.rename(columns = {'Tickets' : 'FSE', 'Customer ID' : 'Unique Accounts'})

In [148]:
logitix = pd.read_excel("C:\\Users\\riffere\\OneDrive - Florida Panthers\\Documents\\logitix_2324.xlsx")

In [149]:
dffinal = pd.concat([df,df2,logitix])
dffinal = dffinal[dffinal['Plan Type'].apply(lambda x: len(str(x))>1)]
dffinal = dffinal.reset_index()
goal_list = [1000000,3250000,2250000,4950000,5250000,21700000,7000000,1350000,700000,5050000]
dffinal['bugdet_goal'] = goal_list
dffinal

,index,Plan Type,Gross Revenue,Net Revenue,FSE,Paid Seats,Unique Accounts,Gross Paid ATP,Net Paid ATP,bugdet_goal
0,0,Flex,276570.09,2.589360e+05,0,2899,184,95.401894,89.319076,1000000
1,1,Group,529289.42,4.618690e+05,0,6963,137,76.014566,66.331897,3250000
2,2,Nightly Suites,811271.00,7.864103e+05,0,3797,114,213.661048,207.113590,2250000
3,3,Singles,257035.46,2.499510e+05,0,2777,804,92.558682,90.007562,4950000
4,0,Annual Suites,5532535.60,5.170594e+06,1068,44856,67,123.339923,115.270965,5250000
5,1,Full Season,22326402.66,1.977464e+07,7197,302274,2597,73.861472,65.419576,21700000
6,2,Premier Plan,6461623.00,5.875775e+06,2928,73200,1298,88.273538,80.270150,7000000
7,3,Sponsor,981649.12,8.183090e+05,154,6366,27,154.201872,128.543669,1350000
8,4,Trade,69027.84,6.157200e+04,26,1092,3,63.212308,56.384615,700000
9,0,Secondary,4731333.33,4.421807e+06,2500,111000,6,45.000000,42.000000,5050000


In [150]:
def get_pi_credentials():

    credentials = {
        "username": "panthersinsights@floridapanthers.com",
        "password": Secret.load("panthersinsights-email-password").get()
    }

    return credentials

def load(pi_creds, df: pd.DataFrame) -> None:

    FLA_Sharepoint(**pi_creds).upload_dataframe(
        df = dffinal,
        folder_path = "Resources/2023-24 Temporary Report Data",
        file_name = "union_2324",
        as_csv = True,
        add_log_date = False
    )

    return None 

with ThreadPoolExecutor(1) as pool:
    pi_creds = pool.submit(lambda: get_pi_credentials()).result()
print(pi_creds)
load(pi_creds, dffinal)

{'username': 'panthersinsights@floridapanthers.com', 'password': 'PIPanthers1@'}
/sites/SP-BS/Shared Documents/Data Science/Resources/2023-24 Temporary Report Data/union_2324.csv
